### Customer Churn Project - KNN Approach
### Staying Ahead of Churn: KNN in Action
### Can We Predict Who Leaves? A KNN Approach to Churn
### KNN for Customer Retention                                  |
                                                              |
---------------------------------------------------------------

## Problem Statement
Syriatel faces unpredictable customer churn, that impacts revenue and market share. (**Assumption**)Despite heavy investment in marketing and acquisition, customers leave without warning. In order to identify at-risk customers early, this project will use existing data; usage patterns, subscriptions, and service interactions. All with an aim of early intervention for retention of the customer.

In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('./data/churn.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   state                   3333 non-null   object 
 1   account length          3333 non-null   int64  
 2   area code               3333 non-null   int64  
 3   phone number            3333 non-null   object 
 4   international plan      3333 non-null   object 
 5   voice mail plan         3333 non-null   object 
 6   number vmail messages   3333 non-null   int64  
 7   total day minutes       3333 non-null   float64
 8   total day calls         3333 non-null   int64  
 9   total day charge        3333 non-null   float64
 10  total eve minutes       3333 non-null   float64
 11  total eve calls         3333 non-null   int64  
 12  total eve charge        3333 non-null   float64
 13  total night minutes     3333 non-null   float64
 14  total night calls       3333 non-null   